<a href="https://colab.research.google.com/github/pranshumalik14/ece421-labs-hw/blob/main/labs/lab1/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: Logistic Regression

In this lab, we will be creating a binary classifier using Logistic Regression first implemented using Numpy and then using Tensorflow. The classifier has to be trained on the `notMNIST` dataset, and particularly classify only between the letters `C` (positive class, labelled `1`) and `J` (negative class, labelled `0`). This dataset, representing the ground truth, will be represented as $\mathcal{D}= \{(\mathbf{x}_i, y_i)\}_{i=1}^N$, where there are $N$ datavectors $\mathbf{x}_i \in \mathbb{R}^d$ and labels $y_i \in \{0, 1\}$.

## 1. Logistic Regression with Numpy

We use the following model for computing the probability of a datavector $\mathbf{x}_n\in\mathbb{R}^d$ belonging to a particular class $y_n\in \{0, 1\}$:

$$\hat{p}_\mathbf{w}(y_n\mid \mathbf{x_n}) = \sigma\left((2y_n-1)(\mathbf{w}^\top\mathbf{x}_n + b)\right),$$

given the model parameters $\mathbf{w} \in \mathbb{R}^d$ (wieght) and $b \in \mathbb{R}$ (bias), and the logistic (or sigmoid) function $\sigma(z) = \frac{1}{1+e^{-z}}$.

In [ ]:
import numpy as np

### 1.1 Loss Function and Gradient 

We will use the regularized loss function (in-sample error) for minimization while training over the dataset:

$$\begin{align}
E_{\text{in}, \lambda}(\mathbf{w}) &= \lambda||\mathbf{w}||^2 + \frac{1}{N}\sum_{n=1}^N -\log\left(\hat{p}_\mathbf{w}(y_n\mid \mathbf{x}_n)\right)\\
&= \lambda||\mathbf{w}||^2 + \frac{1}{N}\sum_{n=1}^N \big[-I(y_n=1)\log(\hat{p}_\mathbf{w}(1\mid \mathbf{x}_n)) -I(y_n=0)\log(\hat{p}_\mathbf{w}(0\mid \mathbf{x}_n))\big] \quad \triangleright \text{since } y_n \text{ only has two possibilities}\\
&= \lambda||\mathbf{w}||^2 + \frac{1}{N}\sum_{n=1}^N \big[-y_n\log(\hat{p}_\mathbf{w}(1\mid \mathbf{x}_n)) -(1-y_n)\log(1-\hat{p}_\mathbf{w}(1\mid \mathbf{x}_n))\big],
\end{align}
$$
where $\lambda > 0$ is the regularization constant and $I(p)$ is the identifier function defined to be, $I(p) = \begin{cases}1 & \text{predicate } p \text{ is true}\\ 0 & \text{predicate } p \text{ is false}\end{cases}$

The gradient of the loss function is:

$$
\begin{align}
∇_{\mathbf{w}}E_{\text{in}, \lambda}(\mathbf{w}) &= ∇\lambda||\mathbf{w}||^2 + \frac{1}{N}\sum_{n=1}^N ∇\big[-y_n\log(\hat{p}_\mathbf{w}(1\mid \mathbf{x}_n)) -(1-y_n)\log(1-\hat{p}_\mathbf{w}(1\mid \mathbf{x}_n))\big]\\
&=2\lambda\mathbf{w} + \frac{1}{N}\sum_{n=1}^N\big[-y_n\big]\\
&=.
\end{align}
$$

In [ ]:
def loss(w, b, X, ys, reg_lambda):
    sigmoid = lambda z: 1 / (1 + np.exp(-z))

    return sigmoid(X@w + b)

In [ ]:
def grad_loss(w, b, X, ys, reg_lambda):
    return 1.0

### 1.2 Gradient Descent Implementation

In [ ]:
def loadDataGDrive():
    with np.load('/content/drive/MyDrive/Colab Notebooks/notMNIST.npz') as dataset:
        Data, Target = dataset['images'], dataset['labels']
        posClass = 2
        negClass = 9
        dataIndx = (Target==posClass) + (Target==negClass)
        Data = Data[dataIndx]/255.
        Target = Target[dataIndx].reshape(-1, 1)
        Target[Target==posClass] = 1
        Target[Target==negClass] = 0
        np.random.seed(421)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data, Target = Data[randIndx], Target[randIndx]
        trainData, trainTarget = Data[:3500], Target[:3500]
        validData, validTarget = Data[3500:3600], Target[3500:3600]
        testData, testTarget = Data[3600:], Target[3600:]
    return trainData, validData, testData, trainTarget, validTarget, testTarget

x_train, x_valid, x_test, y_train, y_valid, y_test = loadDataGDrive()

### 1.3 Tuning the Learning Rate

### 1.4 Generalization


## 2. Logistic Regression in TensorFlow
